# Prompts templates for OpenAI & Llama - LangChain #2

One of the most fundamental and underrated pieces of langchain is prompting, prompts are what we would send to our LLM to allow them to answer in a formulated response, super useful if we want to do something such as making NPCs speak with a specific accent or creating a song with a specifc twist in mind.

**Chapter 2: Prompts**

Once we have mastered prompts we will be able to dive deeper into chains, memory, and everything else!

1. LangChain uses it's own `PromptTemplate` handler which we will use to feed our LLMs.
2. Using prompts we can make both `system` prompts and `user` prompts.
3. We will go over `few shot learning` and how that can help models perform better.
4. We will also make a `limit` on how many examples a prompt can give to the LLMs.

In [787]:
"""            
Answer the question based on the context below,                 }
if you cannot answer the question using the                     }--->  (Rules) For Our Prompt
provided information answer with "I don't know"                 }

Context: Aurelio AI is an AI development studio                 }
focused on the fields of Natural Language Processing (NLP)      }
and information retrieval using modern tooling                  }--->   Context AI has
such as Large Language Models (LLMs),                           }
vector databases, and LangChain.                                }

Question: Does Aurelio AI do anything related to LangChain?     }--->   User Question

Answer:                                                         }--->   AI Answer
"""

'            \nAnswer the question based on the context below,                 }\nif you cannot answer the question using the                     }--->  (Rules) For Our Prompt\nprovided information answer with "I don\'t know"                 }\n\nContext: Aurelio AI is an AI development studio                 }\nfocused on the fields of Natural Language Processing (NLP)      }\nand information retrieval using modern tooling                  }--->   Context AI has\nsuch as Large Language Models (LLMs),                           }\nvector databases, and LangChain.                                }\n\nQuestion: Does Aurelio AI do anything related to LangChain?     }--->   User Question\n\nAnswer:                                                         }--->   AI Answer\n'

First we need to settup our imports to make sure we have both models we want to test.

In [788]:
import openai

# OpenAI key to use the model
OPENAI_API_KEY = ""
openai.api_key = OPENAI_API_KEY

*ollama pull llama3.2:1b-instruct-fp16* - Use in terminal to download correct model we are using.

In [789]:
from langchain_ollama.chat_models import ChatOllama

# Here we can list a couple of models to choose from.
openai_model = "gpt-4o-mini"
llama_model = "llama3.2:1b-instruct-fp16"

# Choose between which model you would like to use
llm_model = openai_model

Here we can see how the AI will appoach our question, as you can see we have a formulated response, if the context has the answer, then use the context to answer the question, if not, say I don't know, then we also have context and question which are being passed into this similarly to paramaters in a function.

In [790]:
Prompt = """
Answer the question based on the context below,                 
if you cannot answer the question using the                   
provided information answer with "I don't know"

context: {context}

question: {query}
"""
Context = """
Context: Aurelio AI is an AI development studio                 
focused on the fields of Natural Language Processing (NLP)      
and information retrieval using modern tooling                  
such as Large Language Models (LLMs),                           
vector databases, and LangChain. Owned by Andrew Tate  
"""

Firstly we want to create our template we will use for our questions.

In [791]:
from langchain.prompts import ChatPromptTemplate

# Passing the template to the LangChain model.
prompt_template = ChatPromptTemplate.from_template(Prompt)

Now we want to open a LLM where we can ask our questions.

In [792]:
from langchain_community.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=OPENAI_API_KEY)

Apply any question into here!

In [793]:
Question = "Who owns Aurelio AI"

This is where we're converting the message into the correct format for our function.

In [794]:
messages = prompt_template.format_messages(prompt=Prompt, query=Question, context=Context)

In [795]:
if llm_model == llama_model:
    llm = ChatOllama(temperature=0.0, model=llama_model)
    creative_llm = ChatOllama(temperature=0.9, model=llama_model)
elif llm_model == openai_model:
    llm = ChatOpenAI(temperature=0.0, model=openai_model, openai_api_key = OPENAI_API_KEY)
    creative_llm = ChatOpenAI(temperature=0.9, model=openai_model, openai_api_key = OPENAI_API_KEY)
else:
    raise ValueError("You need to apply the correct name to the model you are referring to")

Now we will retrieve our answer and showcase the answer:

In [796]:
answer = llm(messages)
print(answer.content)

Aurelio AI is owned by Andrew Tate.


That works for the basic chatbots, however what if we want to give some questions and answers so that the AI can have a rough idea of what we want it to say?

This section will cover Few Shot Prompt Templates allowing us to be able to go more in depth with our AI

In [797]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

1. Firstly we want to add a example template, this will tell the AI how we would like to format our inputs and outputs, in this case, the questions and answers.
2. Secondly we want to add a prompt template, this again will be used to tell the AI how we would like it to use the examples and answer the questions.

In [798]:
# Define the example template
example_template = """Input: {input}
Output: {output}"""

# Define the prompt template
prompt_template = "Given the following examples, complete the task:"

Here is a list of questions and answers, here you can see that we're trying to match words based on their similarity to other words.

In [799]:
Examples = [
        {"input": "Clouds, Sky, Sun, Space, Planets", "output": "Galaxy"},
        {"input": "Eyes, Face, Human, Animal", "output": "Species"},
    ]

Now we want to make a few shot template where we can feed all this data into.

In [800]:
few_shot_prompt = FewShotPromptTemplate(
    examples=Examples,
    example_prompt=PromptTemplate.from_template(example_template),
    prefix = prompt_template,
    suffix="Input: {input}\nOutput:",
    input_variables=["input"],
    example_separator="\n\n",
)

Here we can carry on with our question to see what it thinks is similar too.

In [801]:
# Format the prompt with input
formatted_prompt = few_shot_prompt.format(input="Fitness, Health, Lifestyle")
print(formatted_prompt)

Given the following examples, complete the task:

Input: Clouds, Sky, Sun, Space, Planets
Output: Galaxy

Input: Eyes, Face, Human, Animal
Output: Species

Input: Fitness, Health, Lifestyle
Output:


Now we need to run this through our model and see what the output is!

In [802]:
from langchain.schema import HumanMessage

In [803]:
output = creative_llm(messages=[HumanMessage(content=formatted_prompt)])
print(output.content)

Wellness


However that is considered the old way of doing few shot prompts, here below we can see a newer way which is way easier.

In [804]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [805]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
# This is our few shot template used to feed our examples into the LLM
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=Examples,
)
# Here we can view the format of our content down below
print(few_shot_prompt.format())

Human: Clouds, Sky, Sun, Space, Planets
AI: Galaxy
Human: Eyes, Face, Human, Animal
AI: Species


In [806]:
# Here we are formatting our final prompt for the AI to use, take note that the input needs to be the same keyword as the input below.
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a poem writer."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a poem writer.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'Clouds, Sky, Sun, Space, Planets', 'output': 'Galaxy'}, {'input': 'Eyes, Face, Human, Animal', 'output': 'Species'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [807]:
from langchain.chains import LLMChain

chain = LLMChain(prompt=final_prompt, llm=creative_llm)

chain.invoke({"input": "Fitness, Health, Lifestyle"})

{'input': 'Fitness, Health, Lifestyle', 'text': 'Wellness'}

Now we want to dive into Chain-Of-Thought prompting, for this there is no direct prompt template however instead what we do is we set the template up to make the AI talk to us about how it will solve the problem, by going through each step rather then just rushing straight to the answer.

In [808]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

Here we will settup the step by step rule to enable chain of thought prompting.

In [809]:
# Define the chain-of-thought prompt template
template = """
Question: {question}

First, list systematically and in detail all the problems in this question
that need to be solved before we can arrive at the correct answer.
Then, solve each sub problem using the answers of previous problems
and reach a final solution

Output: Well the right answer is... """

In [810]:
# Define the chain-of-thought prompt template
template2 = """
Question: {question}

Do not use any chain-of-thought processes to answer the question

Output: Well the wrong answer is... """

Then the rest is the same as a simple prompt, where we feed a basic question into the LLM and it will give us a answer.

In [811]:
prompt = PromptTemplate(template=template, input_variables=["question"])
prompt2 = PromptTemplate(template=template2, input_variables=["question"])

In [812]:
# Load the question-answering chain with the chain-of-thought prompt
chain = LLMChain(prompt = prompt, llm = llm)
# Load the question-answering chain with the chain-of-thought prompt
chain2 = LLMChain(prompt = prompt2, llm = llm)

In [813]:
# Ask a question
question1 = "How many keystrokes are needed to type the numbers from 1 to 500? Answer Choices:" # 1392 is the correct answer
question2 = "James has 7 apples, he eats 4 and is given an additional 19 apples, James gives 15 apples to Josh, and Josh gives James 2 apples, how many apples does James have?" 

In [814]:
result = chain.run(question1)

print(result)

To determine how many keystrokes are needed to type the numbers from 1 to 500, we need to break down the problem into several sub-problems. Here are the steps we need to take:

### Sub-Problem 1: Count the Number of Digits in Each Range
We need to categorize the numbers from 1 to 500 based on how many digits they have:

1. **1-digit numbers (1 to 9)**: 
   - There are 9 numbers (1, 2, 3, 4, 5, 6, 7, 8, 9).
   - Each of these numbers has 1 digit.

2. **2-digit numbers (10 to 99)**: 
   - There are 90 numbers (10 to 99).
   - Each of these numbers has 2 digits.

3. **3-digit numbers (100 to 499)**: 
   - There are 400 numbers (100 to 499).
   - Each of these numbers has 3 digits.

4. **The number 500**: 
   - This is a single number with 3 digits.

### Sub-Problem 2: Calculate the Total Keystrokes for Each Range
Now we will calculate the total number of keystrokes for each range of numbers:

1. **Total keystrokes for 1-digit numbers**:
   - Number of 1-digit numbers = 9
   - Total keystr

In [815]:
result = chain2.run(question1)

print(result)

Well the wrong answer is 1500.


As you can see with the amount of steps involved without using a step by step guide, the AI can step over important information that would help it achieve the correct results, however it's important to note that without a super long question the AI can solve alot of these issues without using chain prompting, however it's just in these certain instances where chain prompting can be very useful.